In [2]:
import joblib
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%matplotlib inline

## TIPO_CONTRATACION tiene obra publica y servicios op, cuál es la diferencia?

## Registro Único de Proveedores y Contratistas.xlsx

In [2]:
# hay RFCs nulos

df_registro = pd.read_excel('../data/raw/Registro Único de Proveedores y Contratistas.xlsx')
df_registro.loc[:, 'FECHA_INSCRIPCION_RUPC'] = pd.to_datetime(df_registro.FECHA_INSCRIPCION_RUPC, yearfirst=False)
print(df_registro.shape)
# df_registro.to_csv(
#     '../data/processed/Registro_Unico_Proveedores_y_Contratistas.csv', encoding='utf-8', index=False, quoting=1
# )

df_registro.head()

(8649, 16)


,FOLIO_RUPC,RFC,NOMBRE_EMPRESA,PAIS,ENTIDAD_FEDERATIVA,ESTRATIFICACION,TIPO_USUARIO,SECTOR,GIRO,CONTRATOS,FECHA_INSCRIPCION_RUPC,CONTRATOS_EVALUADOS_ADQ,GRADO_CUMPLIMIENTO_ADQ,CONTRATOS_EVALUADOS_OP,GRADO_CUMPLIMIENTO_OP,SITIO_WEB
0,434,GFE061004F65,GRUPO FARMACOS ESPECIALIZADOS SA DE CV,MX,MX-DF,No MIPYME,Proveedor,Comercio,"COMPRA, VENTA, FABRICACION, DISTRIBUCION, IMPO...",2606,2013-11-03,395.0,98.02,NaN,NaN,NaN
1,579,FMA9301181B1,"FARMACEUTICOS MAYPO, S.A. DE C.V.",MX,MX-DF,No MIPYME,Proveedor,Servicio,"COMPRA-VENTA, IMPORTACIÓN, EXPORTACIÓN, DISTRI...",2578,2013-11-03,215.0,98.19,NaN,NaN,NaN
2,654,DIM010319S79,DISTRIBUIDORA INTERNACIONAL DE MEDICAMENTOS Y ...,MX,MX-JAL,No MIPYME,Proveedor,NaN,"A)Comprar, vender, consignar, distribuir, actu...",1685,2012-08-28,282.0,96.01,NaN,NaN,NaN
3,263,LPI830527KJ2,LABORATORIOS PISA S.A. DE C.V.,MX,MX-JAL,No MIPYME,Proveedor,NaN,"Farmaceutico.\n\nProduccion, venta, compra y d...",1443,2012-08-28,101.0,99.61,NaN,NaN,www.pisafarmaceutica.com.mx
4,3904,INF891031LT4,"INFRA, S.A. DE C.V.",MX,MX-MEX,No MIPYME,Proveedor,NaN,"COMPRA, VENTA, FABRICACION Y DISTRIBUCION DE G...",1256,2012-05-24,155.0,98.50,NaN,NaN,www.infra.com.mx


## PAAASOP 2012-2016

In [2]:

## Revisar el documento con columnas porque no todos los años tienen las mismas cols

year_files = [(i, f'../data/raw/PAAASOP {i}.csv') for i in range(2012, 2017)]

dfs = []
for year, file in year_files:
    df = pd.read_csv(file, encoding='iso-8859-1')
    df = df.rename(columns={'VALOR_ MIPyMES': 'VALOR_MIPYMES'})
    df = df.assign(Year=year)
    dfs.append(df)

df_PAAASOP = pd.concat(dfs, axis=0, ignore_index=True)
print(df_PAAASOP.shape)

# No estoy seguro si este mapeo es correcto, falta verificar

caracter_map = {'N': 'Nacional', 'I': 'Internacional bajo TLC', 'A': 'Internacional abierta'}
def replace_vals(x):
    if x in caracter_map:
        return caracter_map[x]
    return x

df_PAAASOP.loc[:, 'CARACTER_PROCEDIMIENTO'] = df_PAAASOP.CARACTER_PROCEDIMIENTO.map(replace_vals)

# Columnas AÑOS_PLURIANUALES viene muy sucia
# CLAVE_CUCOP tiene 8 números
# CLAVE_PROGRAMA_FEDERAL viene muy vacio o 'no aplica'
# la ENTIDAD_FEDERATIVA no viene homologada, vienen nombres y números
# PLURIANUAL trae 'si', 'no' y también trae números (0, 1, 2, 7)
# Se tienen que revisar los PORCENTAJE_TRIMESTRE

# Homologación de TIPO_CONTRATACION

df_PAAASOP.loc[
    df_PAAASOP.TIPO_CONTRATACION.str.startswith('Servicios Obra P') & (~df_PAAASOP.TIPO_CONTRATACION.isnull()),
    'TIPO_CONTRATACION'
] = 'Obra Publica'

df_PAAASOP.loc[
    :, 'TIPO_CONTRATACION'
] = df_PAAASOP.TIPO_CONTRATACION.str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.lower()


# Homologaciónde tipo_procedimiento

# Faltan
proc_map = {'AD': 'Adjudicacion Directa', 'LP': 'Licitacion Publica', 'I3P':'Invitacion 3 Participantes'}

def replace_vals(x):
    if x in proc_map:
        return proc_map[x]
    return x

df_PAAASOP.loc[:, 'TIPO_PROCEDIMIENTO'] = df_PAAASOP.TIPO_PROCEDIMIENTO.map(replace_vals)

df_PAAASOP.loc[
    :, 'TIPO_PROCEDIMIENTO'
] = df_PAAASOP.TIPO_PROCEDIMIENTO.str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.lower()

# No sé que es UNIDAD_MEDIDA

# Cuál es la diferencia entre todos los 'VALOR_*'?

# df_PAAASOP.to_csv('../data/processed/PAAASOP_2012_2016.csv', index=False, quoting=1, encoding='utf-8')

print(df_PAAASOP.shape)
df_PAAASOP.head()

(2890029, 23)
(2890029, 23)


,AÑOS_PLURIANUALES,CANTIDAD,CARACTER_PROCEDIMIENTO,CLAVE_CUCOP,CLAVE_PROGRAMA_FEDERAL,DEPENDENCIA_ENTIDAD,DESCRIPCION_CUCOP,ENTIDAD_FEDERATIVA,PLURIANUAL,PORCENTAJE_TRIMESTRE1,PORCENTAJE_TRIMESTRE2,PORCENTAJE_TRIMESTRE3,PORCENTAJE_TRIMESTRE4,PRODUCTO,TIPO_CONTRATACION,TIPO_GOBIERNO,TIPO_PROCEDIMIENTO,UNIDAD_MEDIDA,VALOR_ANUAL_PLURIANUAL,VALOR_ESTIMADO,VALOR_MIPYMES,VALOR_NCTLC,Year
0,0,32.0,Nacional,35200003,NaN,Administración del Patrimonio de la Beneficenc...,NaN,9,0,NaN,NaN,NaN,NaN,Mobiliario y equipo de administracion (manteni...,servicios,NaN,adjudicacion directa,97,0,"11,280.00","11,280.00","11,280.00",2012
1,0,6.0,Nacional,29100063,NaN,Administración del Patrimonio de la Beneficenc...,NaN,9,0,NaN,NaN,NaN,NaN,Desarmador,adquisiciones,NaN,adjudicacion directa,51,0,"1,169.28","1,169.28","1,169.28",2012
2,0,7.0,Nacional,29100063,NaN,Administración del Patrimonio de la Beneficenc...,NaN,9,0,NaN,NaN,NaN,NaN,Desarmador,adquisiciones,NaN,adjudicacion directa,51,0,"1,680.84","1,680.84","1,680.84",2012
3,0,2.0,Nacional,29100177,NaN,Administración del Patrimonio de la Beneficenc...,NaN,9,0,NaN,NaN,NaN,NaN,Martillo uña,adquisiciones,NaN,adjudicacion directa,131,0,139.78,139.78,139.78,2012
4,0,3.0,Nacional,29100142,NaN,Administración del Patrimonio de la Beneficenc...,NaN,9,0,NaN,NaN,NaN,NaN,Llave española,adquisiciones,NaN,adjudicacion directa,51,0,"2,373.36","2,373.36","2,373.36",2012


## Directorio Unidades Compradoras Gobierno Federal

In [9]:
# clave UC es de 9 caracteres

df_dir_fed = pd.read_excel('../data/raw/Directorio Unidades Compradoras Gobierno Federal.xls')
print(df_dir_fed.shape)
df_dir_fed.head()

(1930, 12)


,Nombre UC en CompraNet,Clave UC en CompraNet,Pertenece al,Dependencia o Entidad,Siglas,Página Web,Dirección,CP,Entidad Federativa,Delegación o municipio,Telefono de la UC,Nombre del Responsable
0,AFSEDF-ADMINISTRACIÓN FEDERAL DE SERVICIOS EDU...,011C00999,APF,Administración Federal de Servicios Educativos...,AFSEDF,www.sep.gob.mx,Río Rhin 9 Piso 8,06500,Ciudad de México,Cuauhtémoc,36017100 ext. 40302,Ricardo Rafael Nieto Hernández
1,API-Altamira-Departamento de Concursos y Contr...,009J2Y001,APF,"Administración Portuaria Integral de Altamira,...",API-Altamira,www.puertoaltamira.com.mx,"Calle Rios Tamesi Km 0 800, lado sur, Colonia ...",89603,Tamaulipas,Altamira,18332606060,Alberto Trejo Saldaña
2,API-Altamira-DEPARTAMENTO DE RECURSOS MATERIAL...,009J2Y002,APF,"Administración Portuaria Integral de Altamira,...",API-Altamira,www.puertoaltamira.com.mx,RIO TAMESI KM 0 800 LADO SUR COLONIA PUERTO IN...,89603,Tamaulipas,Altamira,8332606060,Denisse Ramirez Diaz
3,API-Coatzacoalcos-Coatzacoalcos Jefatura de Pr...,009J3F001,APF,Administración Portuaria Integral de Coatzacoa...,API-Coatzacoalcos,http://www.puertocoatzacoalcos.com.mx/,Interior del Recinto Fiscal s/n antes zona franca,96400,Veracruz,Coatzacoalcos,019212110270,José Manuel Kin Galindo
4,API-Coatzacoalcos-Administración Portuaria Int...,009J3F002,APF,Administración Portuaria Integral de Coatzacoa...,API-Coatzacoalcos,www.puertocoatzacoalcos.com.mx,Interior del Recinto Fiscal Zona Franca S/N,96400,Veracruz,Coatzacoalcos,9212110270,Bernardino Guzmán Zavala


## Directorio Unidades Compradoras Gobierno Estatal

In [15]:
# hay un Nan en CP
# Sería bueno homologar todos los responsables y los nombres

df_dir_est = pd.read_excel('../data/raw/Directorio Unidades Compradoras Gobierno Estatal.xls')
print(df_dir_est.shape)
df_dir_est.head()

(1114, 12)


,Nombre UC en CompraNet,Clave UC en CompraNet,Pertenece al,Dependencia o Entidad,Siglas,Página Web,Dirección,CP,Entidad Federativa,Delegación o municipio,Telefono de la UC,Nombre del Responsable
0,AGS-Universidad Tecnológica de Aguascalientes-...,901041992,GE,_Gobierno del Estado de Aguascalientes,AGS,www.utags.edu.mx,BLVD. JUAN PABLO II No. 1302 FRACC. EXHACIENDA...,20200.0,Aguascalientes,Aguascalientes,014499105000,MONICA GARCIA PLASCENCIA
1,AGS-Instituto del Agua del Estado-COORDINACION...,901030992,GE,_Gobierno del Estado de Aguascalientes,AGS,www.aguascalientes.gob.mx/inagua,C.18 DE MARZO NO.98 FRACC. LAS HADAS,20140.0,Aguascalientes,Aguascalientes,449 9102586,ROGERIO GAMALIEL RAMIREZ RANGEL
2,AGS-Secretaría de Obras Públicas-Secretaria de...,901006994,GE,_Gobierno del Estado de Aguascalientes,AGS,www.jesusmaria.gob.mx,"Emiliano Zapata No. 109, Zona Centro",20900.0,Aguascalientes,Jesús María,01(449) 9650088,DAVID FERNANDO MORAN CABELLO
3,AGS-Fondo de Fomento Agropecuario del Estado-F...,901058977,GE,_Gobierno del Estado de Aguascalientes,AGS,www.aguascalientes.gob.mx,Av. de la Convencion Ote No. 102; Col. Del Tra...,20180.0,Aguascalientes,Aguascalientes,4499102596,SERGIO LEON JAZZO
4,AGS-Secretaría de la Juventud-SECRETARIA DE LA...,901025974,GE,_Gobierno del Estado de Aguascalientes,AGS,www.aguascalientes.gob.mx,Av. Revolución s/n Col Ferronales,20180.0,Aguascalientes,Aguascalientes,4499102141,SILVIA KARENI REYES ROSAS


## Directorio Unidades Compradoras Gobierno Municipal

In [19]:
# No sé porque las dependencias empiezan con '_'
# Sería bueno homologar todos los responsables y los nombres

df_dir_mun = pd.read_excel('../data/raw/Directorio Unidades Compradoras Gobierno Municipal.xls')
print(df_dir_mun.shape)
df_dir_mun.head()

(2137, 12)


,Nombre UC en CompraNet,Clave UC en CompraNet,Pertenece al,Dependencia o Entidad,Siglas,Página Web,Dirección,CP,Entidad Federativa,Delegación o municipio,Telefono de la UC,Nombre del Responsable
0,AGS-Aguascalientes-Secretaria de Administració...,801001998,GM,_Gobierno Municipal del Estado de Aguascalientes,AGS,www.ags.gob.mx,Plaza de la Patria y calle Colón s/n,20000,Aguascalientes,Aguascalientes,449 9101015,Jóse Antonio Arámbula López
1,AGS-Asientos-MUNICIPIO DE ASIENTOS #801002981,801002981,GM,_Gobierno Municipal del Estado de Aguascalientes,AGS,www.asientos.gob.mx,"PLAZA JUÁREZ No. 3, ZONA CENTRO",20712,Aguascalientes,Asientos,4969674062,VICTOR MANUEL RODRIGUEZ ROMO
2,AGS-El Llano-PRESIDENCIA MUNICIPAL DE EL LLANO...,801010980,GM,_Gobierno Municipal del Estado de Aguascalientes,AGS,www..ayuntamiento@municipiodeelllano.gob.mx,20 DE NOVIEMBRE NO. 103,20330,Aguascalientes,El Llano,4969673587,CEZAR PEDROZA ORTEGA
3,AGS-Aguascalientes-INSTITUTO MUNICIPAL DE RENO...,801001997,GM,_Gobierno Municipal del Estado de Aguascalientes,AGS,www.imuviags.gog.mx,ANTONIO ACEVEDO ESCOBEDO 103 ZONA CENTRO,20000,Aguascalientes,Aguascalientes,449 9101010 EXT 3096,LUIS HUMBERTO HUERTA HERNANDEZ
4,AGS-Aguascalientes-MUNICIPIO AGUASCALIENTES #8...,801001999,GM,_Gobierno Municipal del Estado de Aguascalientes,AGS,www.ags.gob.mx,"AV. AGUASCALIENTES SUR 2615, FRACC. JARDINES D...",20290,Aguascalientes,Aguascalientes,4499789201 ext 140,HUMBERTO CRUZ HERNANDEZ


## Procedimmientos de contratacion

In [30]:
def get_df(name):
    df = pd.read_excel('{0}'.format(name))
    df.loc[:, 'PROC_F_PUBLICACION'] = pd.to_datetime(df.PROC_F_PUBLICACION, yearfirst=True)
    df.loc[:, 'FECHA_APERTURA_PROPOSICIONES'] = pd.to_datetime(df.FECHA_APERTURA_PROPOSICIONES, yearfirst=True)
    df.loc[:, 'EXP_F_FALLO'] = pd.to_datetime(df.EXP_F_FALLO, yearfirst=True)
    df.loc[:, 'FECHA_CELEBRACION'] = pd.to_datetime(df.FECHA_CELEBRACION, yearfirst=True)
    df.loc[:, 'FECHA_INICIO'] = pd.to_datetime(df.FECHA_INICIO, yearfirst=True)
    df.loc[:, 'FECHA_FIN'] = pd.to_datetime(df.FECHA_FIN, yearfirst=True)

    df.loc[
        :, 'TIPO_CONTRATACION'
    ] = df.TIPO_CONTRATACION.str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.lower()
    df.loc[
        :, 'TIPO_PROCEDIMIENTO'
    ] = df.TIPO_PROCEDIMIENTO.str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.lower()
    df.loc[
        :, 'RESPONSABLE'
    ] = df.RESPONSABLE.str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.lower()
    df.loc[
        :, 'DEPENDENCIA'
    ] = df.DEPENDENCIA.str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.lower()
    return df

file_names = [
    '../data/raw/Procedimiento de Contratación 2010-2012.xlsx',
    '../data/raw/Procedimiento de Contratación 2013.xlsx',
    '../data/raw/Procedimiento de Contratación 2014.xlsx',
    '../data/raw/Procedimiento de Contratación 2015.xlsx',
    '../data/raw/Procedimiento de Contratación 2016.xlsx'
]

df_procedimientos = joblib.Parallel(n_jobs=-1)(joblib.delayed(get_df)(name) for name in file_names)
df_procedimientos = pd.concat(df_procedimientos, axis=0, ignore_index=True)
df_procedimientos = df_procedimientos.loc[(df_procedimientos.FECHA_INICIO.dt.year > 2011), :]
df_procedimientos.loc[:, 'CLAVEUC'] = df_procedimientos.CLAVEUC.astype(str)

# RAMO tiene números, no sé a qué se refiere
# Para qué sirve el código de contrato


df_procedimientos.head()

,ANUNCIO,APORTACION_FEDERAL,ARCHIVADO,CARACTER,CLAVEUC,CLAVE_CARTERA_SHCP,CLAVE_PROGRAMA,CODIGO_CONTRATO,CODIGO_EXPEDIENTE,COMPRA_CONSOLIDADA,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,CUENTA_ADMINISTRADA_POR,C_EXTERNO,DEPENDENCIA,ESTATUS_CONTRATO,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,ESTRATIFICACION_MUC,EXP_F_FALLO,FECHA_APERTURA_PROPOSICIONES,FECHA_CELEBRACION,FECHA_FIN,FECHA_INICIO,FOLIO_RUPC,FORMA_PROCEDIMIENTO,GOBIERNO,IDENTIFICADOR_CM,IMPORTE_CONTRATO,MONEDA,NOMBRE_DE_LA_UC,NUMERO_PROCEDIMIENTO,ORGANISMO,PLANTILLA_EXPEDIENTE,PLURIANUAL,PROC_F_PUBLICACION,PROVEEDOR_CONTRATISTA,RAMO,RESPONSABLE,SIGLAS,SIGLAS_PAIS,TIPO_CONTRATACION,TIPO_PROCEDIMIENTO,TITULO_CONTRATO,TITULO_EXPEDIENTE
0,NaN,NaN,No,NaN,901024986,NaN,NaN,208993,271313,0.0,0.0,0.0,PoC,NaN,_gobierno del estado de aguascalientes,Expirado,HABILITADO,Micro,No MIPYME,2012-10-23,2012-10-23 11:30:00,NaT,2012-12-31,2012-10-23,NaN,NaN,GE,NaN,512930.40,MXN,AGS-Instituto de Educación de Aguascalientes-I...,IA-901024986-N44-2012,NaN,Z15122015 04. Invitación a cuando menos tres N...,0.0,2012-10-17 14:51:00,RAUL EUGENIO ALVAREZ CARRASCO,NaN,antonio ruben cienfuegos benavides,AGS,MX,adquisiciones,invitacion a cuando menos 3 personas,N44 MOB ETC,ADQUISICIÓN DE MOBILIARIO
1,NaN,NaN,No,NaN,901024986,NaN,NaN,208992,271313,0.0,0.0,0.0,PoC,NaN,_gobierno del estado de aguascalientes,Expirado,HABILITADO,No MIPYME,No MIPYME,2012-10-23,2012-10-23 11:30:00,NaT,2012-10-31,2012-10-23,NaN,NaN,GE,NaN,1887409.40,MXN,AGS-Instituto de Educación de Aguascalientes-I...,IA-901024986-N44-2012,NaN,Z15122015 04. Invitación a cuando menos tres N...,0.0,2012-10-17 14:51:00,MA DE LA LUZ TISCAREÑO ELIAS,NaN,antonio ruben cienfuegos benavides,AGS,MX,adquisiciones,invitacion a cuando menos 3 personas,N44 MOB ETC,ADQUISICIÓN DE MOBILIARIO
2,NaN,NaN,No,NaN,901024986,NaN,NaN,257525,268049,0.0,0.0,0.0,PoC,NaN,_gobierno del estado de aguascalientes,Expirado,HABILITADO,Micro,No MIPYME,2012-10-19,2012-11-12 10:00:00,NaT,2012-12-31,2012-11-12,NaN,NaN,GE,NaN,1892535.82,MXN,AGS-Instituto de Educación de Aguascalientes-I...,IA-901024986-N53-2012,NaN,Z15122015 04. Invitación a cuando menos tres N...,0.0,2012-11-05 14:34:00,PAPELERIA CONSUMIBLES Y ACCESORIOS SA DE CV,NaN,antonio ruben cienfuegos benavides,AGS,MX,adquisiciones,invitacion a cuando menos 3 personas,IEA.DJ.CONT. 1140/2012,IEA PAPELERIA Y CONSUMIBLES
3,NaN,NaN,No,NaN,901024986,NaN,NaN,241054,252725,0.0,0.0,0.0,PoC,NaN,_gobierno del estado de aguascalientes,Expirado,HABILITADO,Pequeña,No MIPYME,NaT,2012-09-07 09:00:00,NaT,2012-12-31,2012-09-11,19761.0,NaN,GE,NaN,24234.00,MXN,AGS-Instituto de Educación de Aguascalientes-I...,AA-901024986-N31-2012,NaN,Z15122015 07. Adjudicación Directa Nacional Ar...,0.0,2012-09-06 08:49:00,TELECOMUNICACIONES MODERNAS SA DE CV,NaN,antonio ruben cienfuegos benavides,AGS,MX,adquisiciones,adjudicacion directa federal,CONSUMIBLES,COORDINACION ESTATAL DE ASESORIA Y SEGUIMIENTO...
5,NaN,NaN,No,NaN,901024986,NaN,NaN,474294,320854,0.0,0.0,0.0,PoC,NaN,_gobierno del estado de aguascalientes,Expirado,HABILITADO,Pequeña,No MIPYME,NaT,2012-12-14 10:00:00,NaT,2012-12-31,2012-12-17,19761.0,NaN,GE,NaN,18700.00,MXN,AGS-Instituto de Educación de Aguascalientes-I...,AA-901024986-N64-2012,NaN,Z15122015 07. Adjudicación Directa Nacional Ar...,0.0,2012-12-12 11:38:00,TELECOMUNICACIONES MODERNAS SA DE CV,NaN,antonio ruben cienfuegos benavides,AGS,MX,adquisiciones,adjudicacion directa federal,Adquisición de equipo de cómputo,EQUIPO DE COMPUTO DESARROLLO EDUCATIVO


In [29]:
df_procedimientos.CARACTER.value_counts(dropna=False)

Nacional                  733184
Internacional              80664
Internacional bajo TLC     20372
NaN                        16119
Internacional abierta      10812
                              18
Name: CARACTER, dtype: int64

In [14]:
df_procedimientos.dtypes

ANUNCIO                                 object
APORTACION_FEDERAL                     float64
ARCHIVADO                               object
CARACTER                                object
CLAVEUC                                 object
CLAVE_CARTERA_SHCP                      object
CLAVE_PROGRAMA                          object
CODIGO_CONTRATO                          int64
CODIGO_EXPEDIENTE                        int64
COMPRA_CONSOLIDADA                     float64
CONTRATO_MARCO                         float64
CONVENIO_MODIFICATORIO                 float64
CUENTA_ADMINISTRADA_POR                 object
C_EXTERNO                              float64
DEPENDENCIA                             object
ESTATUS_CONTRATO                        object
ESTATUS_EMPRESA                         object
ESTRATIFICACION_MPC                     object
ESTRATIFICACION_MUC                     object
EXP_F_FALLO                     datetime64[ns]
FECHA_APERTURA_PROPOSICIONES    datetime64[ns]
FECHA_CELEBRA

In [26]:
df_procedimientos[df_procedimientos.CLAVEUC.str.len().isnull()]

,ANUNCIO,APORTACION_FEDERAL,ARCHIVADO,CARACTER,CLAVEUC,CLAVE_CARTERA_SHCP,CLAVE_PROGRAMA,CODIGO_CONTRATO,CODIGO_EXPEDIENTE,COMPRA_CONSOLIDADA,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,CUENTA_ADMINISTRADA_POR,C_EXTERNO,DEPENDENCIA,ESTATUS_CONTRATO,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,ESTRATIFICACION_MUC,EXP_F_FALLO,FECHA_APERTURA_PROPOSICIONES,FECHA_CELEBRACION,FECHA_FIN,FECHA_INICIO,FOLIO_RUPC,FORMA_PROCEDIMIENTO,GOBIERNO,IDENTIFICADOR_CM,IMPORTE_CONTRATO,MONEDA,NOMBRE_DE_LA_UC,NUMERO_PROCEDIMIENTO,ORGANISMO,PLANTILLA_EXPEDIENTE,PLURIANUAL,PROC_F_PUBLICACION,PROVEEDOR_CONTRATISTA,RAMO,RESPONSABLE,SIGLAS,SIGLAS_PAIS,TIPO_CONTRATACION,TIPO_PROCEDIMIENTO,TITULO_CONTRATO,TITULO_EXPEDIENTE


In [24]:
type(df_procedimientos.loc[247424, 'CLAVEUC'])

int

In [20]:
df_procedimientos[df_procedimientos.CLAVEUC == '']

,ANUNCIO,APORTACION_FEDERAL,ARCHIVADO,CARACTER,CLAVEUC,CLAVE_CARTERA_SHCP,CLAVE_PROGRAMA,CODIGO_CONTRATO,CODIGO_EXPEDIENTE,COMPRA_CONSOLIDADA,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,CUENTA_ADMINISTRADA_POR,C_EXTERNO,DEPENDENCIA,ESTATUS_CONTRATO,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,ESTRATIFICACION_MUC,EXP_F_FALLO,FECHA_APERTURA_PROPOSICIONES,FECHA_CELEBRACION,FECHA_FIN,FECHA_INICIO,FOLIO_RUPC,FORMA_PROCEDIMIENTO,GOBIERNO,IDENTIFICADOR_CM,IMPORTE_CONTRATO,MONEDA,NOMBRE_DE_LA_UC,NUMERO_PROCEDIMIENTO,ORGANISMO,PLANTILLA_EXPEDIENTE,PLURIANUAL,PROC_F_PUBLICACION,PROVEEDOR_CONTRATISTA,RAMO,RESPONSABLE,SIGLAS,SIGLAS_PAIS,TIPO_CONTRATACION,TIPO_PROCEDIMIENTO,TITULO_CONTRATO,TITULO_EXPEDIENTE


In [ ]:
df_procedimientos.CLAVEUC.str.len()

In [13]:
df_procedimientos[df_procedimientos.CLAVEUC.isnull()]

,ANUNCIO,APORTACION_FEDERAL,ARCHIVADO,CARACTER,CLAVEUC,CLAVE_CARTERA_SHCP,CLAVE_PROGRAMA,CODIGO_CONTRATO,CODIGO_EXPEDIENTE,COMPRA_CONSOLIDADA,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,CUENTA_ADMINISTRADA_POR,C_EXTERNO,DEPENDENCIA,ESTATUS_CONTRATO,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,ESTRATIFICACION_MUC,EXP_F_FALLO,FECHA_APERTURA_PROPOSICIONES,FECHA_CELEBRACION,FECHA_FIN,FECHA_INICIO,FOLIO_RUPC,FORMA_PROCEDIMIENTO,GOBIERNO,IDENTIFICADOR_CM,IMPORTE_CONTRATO,MONEDA,NOMBRE_DE_LA_UC,NUMERO_PROCEDIMIENTO,ORGANISMO,PLANTILLA_EXPEDIENTE,PLURIANUAL,PROC_F_PUBLICACION,PROVEEDOR_CONTRATISTA,RAMO,RESPONSABLE,SIGLAS,SIGLAS_PAIS,TIPO_CONTRATACION,TIPO_PROCEDIMIENTO,TITULO_CONTRATO,TITULO_EXPEDIENTE


In [8]:
df_procedimientos.FECHA_CELEBRACION.value_counts(dropna=False)

NaT                    440807
2016-01-27 00:00:00      1866
2016-01-01 00:00:00      1648
2015-01-01 00:00:00      1560
2012-07-13 00:00:00      1516
2014-11-14 00:00:00      1457
2016-03-01 00:00:00      1417
2016-07-01 00:00:00      1351
2016-04-01 00:00:00      1193
2013-01-01 00:00:00      1056
2013-11-29 00:00:00       992
2016-10-03 00:00:00       987
2015-04-01 00:00:00       963
2014-01-01 00:00:00       944
2016-05-02 00:00:00       934
2013-02-28 00:00:00       930
2015-11-30 00:00:00       929
2013-04-01 00:00:00       920
2015-02-27 00:00:00       914
2015-07-01 00:00:00       885
2014-02-28 00:00:00       869
2013-03-01 00:00:00       866
2016-02-29 00:00:00       847
2012-03-01 00:00:00       842
2012-01-01 00:00:00       807
2012-01-16 00:00:00       796
2016-09-01 00:00:00       785
2016-11-30 00:00:00       783
2016-06-30 00:00:00       770
2015-03-13 00:00:00       766
2014-04-01 00:00:00       751
2013-12-31 00:00:00       749
2015-01-16 00:00:00       744
2016-07-18

In [7]:
df_PAAASOP.TIPO_PROCEDIMIENTO.unique()

array(['AD', 'LP', 'I3P', 'Adjudicacion Directa', 'Licitacion Publica',
       'Invitacion 3 Participantes',
       'Invitación a cuando menos 3 personas - Asociaciones Público Privada',
       'Adjudicación Directa - Asociaciones Público Privada',
       'Licitación Pública', 'Adjudicación Directa',
       'Invitación a cuando menos tres personas',
       'Invitación a cuando menos tres personas - Asociaciones Público Privada'], dtype=object)

In [6]:
df_PAAASOP.TIPO_PROCEDIMIENTO.value_counts(dropna=False)

Licitacion Publica                                                        905030
Adjudicacion Directa                                                      670776
Licitación Pública                                                        314419
LP                                                                        304102
AD                                                                        211880
Adjudicación Directa                                                      195245
Invitacion 3 Participantes                                                181237
I3P                                                                        58291
Invitación a cuando menos tres personas                                    48382
Adjudicación Directa - Asociaciones Público Privada                          538
Invitación a cuando menos 3 personas - Asociaciones Público Privada          116
Invitación a cuando menos tres personas - Asociaciones Público Privada        13
Name: TIPO_PROCEDIMIENTO, dt

In [15]:
# df_PAAASOP.loc[
#     (df_PAAASOP.CARACTER_PROCEDIMIENTO == 'A') |
#     (df_PAAASOP.CARACTER_PROCEDIMIENTO == 'Internacional abierta'), ['CLAVE_CUCOP', 'CARACTER_PROCEDIMIENTO', 'PRODUCTO']].sort_values(
# 'CLAVE_CUCOP')